In [5]:
import hmmlearn.hmm as hmm
import numpy as np
import itertools
import pandas as pd
from scipy.stats import norm
from scipy.optimize import minimize
from docx import Document
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.dates as mdates
from datetime import datetime
import time

def read_clusters(file_path):
    doc=Document(file_path)
    cluster_num=len(doc.tables[3].rows)-1
    clusters=[]
    for i in range(1,len(doc.tables[3].rows)):
        clusters.append(float(doc.tables[3].cell(i,1).text))
    return (cluster_num,clusters)
 
    
def read_dataset(file_path,require_time=None):
    f=open(file_path,'r')
    value=[]
    time_line=[]
    line=f.readline()
    while line:
        data=line.split(' ')
        value.append(int(data[1][0:len(data[1])-1]))
        time_line.append(int(data[0]))
        line=f.readline()
    f.close()
    if require_time==None:
        return value
    else:
        return value,time_line
    
       

class factorial_hmm(object):
    def __init__(self,HMMs_num,house):
        self.HMMs_num=HMMs_num   # Nombre de chaînes de Markov cachées, c'est-à-dire le nombre total d'appareils
        self.house=house         # L'ensemble des données correspond au numéro de série de la pièce
        self.HMMs_para={}        # Dictionnaire contenant chaque paramètre HMM
        self.states_combination=[]  # Chaque tuple d'état hmm, de la forme ('1', '2', '3', '4',...) La longueur de ce n-uplet est le nombre d'états cachés hmm
        self.startprob_=None      # fhmm distribution de l'état initial
        self.transmat_=None       # Matrice de transfert d'état
        self.means_=None          # fhmm matrice moyenne de la densité de probabilité de l'état observé
        self.covars_=None         # matrice de covariance de la densité de probabilité de l'état observé
        self.n_components=None    # Nombre total d'états cachés #fhmm
        self.states_frame=None    # Tableau d'identification des états cachés pour chaque hmm
        self.observe_frame=None   # Tableaux de décomposition de la puissance active des charges hmm pour chaque barre
        self.aggregate_frame=None # Tableaux de données échantillonnées de la puissance active totale synthétisée sur une période donnée
        self.true_value_frame_dic={} # Dictionnaire composé d'un tableau de données d'échantillonnage actives réelles pour chaque appareil sur une période donnée, indexées par canal.
         
    def get_HMMs_para(self):
        '''
        En utilisant les données d'échantillonnage actif de chaque appareil électrique et le nombre d'états pour chaque appareil obtenu à partir du clustering      
        précédent, nous apprenons et obtenons les paramètres HMM pour chaque canal. Les résultats sont stockés dans un dictionnaire appelé self.HMMs_para o
        ù la clé est le canal et la valeur est une liste contenant successivement pi, A, la moyenne, et la covariance.
        
        '''
        for j in range(2,self.HMMs_num+2):
            cluster_path='C:/Users/lenovo/Desktop/domestic UK2015/house'+str(self.house)+'/channel_'+str(j)+'.docx'
            states_num,means=read_clusters(cluster_path)
            dataset_path='C:/Users/lenovo/Desktop/house'+str(self.house)+'/channel_'+str(j)+'.dat'
            sample_value=read_dataset(dataset_path)
            HMM_model=hmm.GaussianHMM(n_components=states_num, covariance_type='full')
            HMM_model.fit(np.array([sample_value]).reshape(-1,1))
            self.HMMs_para['channel_'+str(j)]=[HMM_model.startprob_,HMM_model.transmat_,\
                           HMM_model.means_,HMM_model.covars_]
            
    def compute_FHMM_para(self):
        '''
        Appel à la fonction pour calculer les différents paramètres de chaque FHMM et stocker les résultats. 
        Le nombre de colonnes pour pi, l'ordre de A, le nombre d'éléments pour les moyennes et les covariances s
        ont tous égaux au nombre d'états cachés, c'est-à-dire self.n_components. L'ordre d'agencement de tous les paramètres correspond à 
        l'ordre d'agencement des combinaisons d'états dans la variable self.states_combination.

        '''
        self.startprob_=self.compute_pi_fhmm()
        self.transmat_=self.compute_A_fhmm()
        self.means_=self.compute_means_fhmm()
        self.covars_=self.compute_covars_fhmm()
        self.n_components=self.compute_n_components()
        
    def compute_pi_fhmm(self):
        '''
        Utiliser le produit de Kronecker pour calculer la matrice pi du modèle FHMM.
        '''
        list_pi=[]
        for j in range(2,len(self.HMMs_para.keys())+2):
            list_pi.append(self.HMMs_para['channel_'+str(j)][0])
        result = list_pi[0]
        for i in range(len(list_pi) - 1):
            result = np.kron(result, list_pi[i + 1])
        return result
    
    def compute_A_fhmm(self):
        '''
       Utiliser le produit de Kronecker pour calculer la matrice A du modèle FHMM.
        '''
        list_A=[]
        for j in range(2,len(self.HMMs_para.keys())+2):
            list_A.append(self.HMMs_para['channel_'+str(j)][1])
        result = list_A[0]
        for i in range(len(list_A) - 1):
            result = np.kron(result, list_A[i + 1])
        return result
    
    def compute_means_fhmm(self):
        '''
        En utilisant la méthode itertools.product, combiner les moyennes de chaque état caché de chaque HMM du FHMM. 
        Ensuite, selon la propriété d'indépendance mutuelle de chaque HMM, additionner ces moyennes pour obtenir la moyenne de la distribution d'observation 
        pour chaque état caché du FHMM.

        En fonction de la liste des moyennes pour chaque HMM, générer une chaîne de chiffres débutant à '1' dans le même ordre que ces moyennes, 
        où la position de la moyenne la plus petite correspond au chiffre '1', et ainsi de suite. En utilisant ensuite la méthode itertools.product 
        pour combiner chaque chaîne de chiffres correspondant à la liste des moyennes, obtenir states_combination. 
        Cette variable de type liste a une longueur égale au nombre d'états cachés du FHMM. Chaque élément représente un état caché du FHMM, 
        et chaque élément est un tuple composé de plusieurs caractères numériques individuels. La longueur du tuple est égale au nombre de chaînes HMM, 
        et cela représente quelles moyennes de quelle chaîne HMM composent cet état caché. L'ordre des éléments dans states_combination correspond à 
        l'ordre des éléments dans pi, A, moyennes et covariances."
        '''
        list_means=[]
        result=[]
        for j in range(2,len(self.HMMs_para.keys())+2):
            list_means.append(self.HMMs_para['channel_'+str(j)][2].reshape(1,-1)[0])
        for i in itertools.product(*list_means):
            result.append(sum(i))
        states_number=[]
        for k in list_means:
            string=''
            sort_means=sorted(k)
            for p in k:
                for t in range(len(sort_means)):
                    if p==sort_means[t]:
                        string+=str(t+1)
                        break
            states_number.append(string)
        for q in itertools.product(*states_number):
            self.states_combination.append(q)
        return np.array([result]).reshape(-1,1)
    
    def compute_covars_fhmm(self):
        '''
        Calculer la matrice de covariance de fhmm
        '''
        list_covars=[]
        result=[]
        for j in range(2,len(self.HMMs_para.keys())+2):
            list_covars.append(self.HMMs_para['channel_'+str(j)][3].reshape(1,-1)[0])
        for i in itertools.product(*list_covars):
            result.append(sum(i))
        return np.array([result]).reshape(-1,1,1)

    def compute_n_components(self):
        '''
        Calculer le nombre d'états cachés fhmm
        '''
        return len(self.states_combination)
    
    def get_aggregate_frame(self,time_start,time_len):
        '''
        Obtenir la somme des valeurs réelles de puissance active de tous les appareils électriques pendant la plage de temps spécifiée par time_range. 
        Renvoyer un tableau (DataFrame) avec pour index les points temporels de cette période et les valeurs représentant la somme des puissances actives réelles. 
        Ce résultat servira de contrainte totale de puissance active pour la désagrégation de la charge électrique ultérieure et pour prédire les états cachés du modèle FHMM.
        '''
        for i in range(2,self.HMMs_num+2):
            dataset_path='C:/Users/lenovo/Desktop/house'+str(self.house)+'/channel_'+str(i)+'.dat'
            sample_value,sample_time=read_dataset(file_path=dataset_path,require_time='yes')
            time_frame=self.get_a_time_frame(time_start,time_len,sample_time,sample_value)
            if i!=2:
                time_frame.index=self.true_value_frame_dic['channel_2'].index
            self.true_value_frame_dic['channel_'+str(i)]=time_frame
    
        aggregate_value=[]
        for j in range(int(time_len)):
            value=0
            for k in range(2,self.HMMs_num+2):
                value+=self.true_value_frame_dic['channel_'+str(k)].values[j,0]
            aggregate_value.append(value)
        self.aggregate_frame=pd.DataFrame(np.array([aggregate_value]).reshape(-1,1),\
                                          index=list(self.true_value_frame_dic['channel_2'].index),columns=['aggregate_value'])
        
    
    def decode(self):
        '''
        En utilisant la somme précédemment calculée des valeurs totales d'énergie active sur une période de temps donnée ainsi que les paramètres synthétiques du FHMM, 
        prédire la séquence d'états cachés du FHMM. Ensuite, selon la correspondance correspondante, organiser les éléments de self.states_combination 
        pour décomposer les états cachés de chaque HMM à chaque instant. Renvoyer un tableau d'états où les index sont les moments dans cette période et 
        les colonnes représentent les canaux.
        '''
        fhmm_model=hmm.GaussianHMM(n_components=self.n_components, covariance_type='full')
        fhmm_model.startprob_=self.startprob_
        fhmm_model.transmat_=self.transmat_
        fhmm_model.means_=self.means_
        fhmm_model.covars_=self.covars_
        logprob,state_sequence=fhmm_model.decode(self.aggregate_frame.values)
        state_sequence_str={}
        for i in range(2,self.HMMs_num+2):
            state_str=[]
            for j in state_sequence:
                state_str.append(int(self.states_combination[j][i-2]))
            state_sequence_str['channel_'+str(i)]=state_str
        self.states_frame=pd.DataFrame(state_sequence_str,index=list(self.aggregate_frame.index))
        
        
    def get_a_time_frame(self,time_start,time_len,full_time_list,full_value_list):
        '''
        Intercepter le point temporel et la valeur d'échantillonnage active 
        correspondante en fonction de l'intervalle de temps, renvoyer une trame.
        '''
        time_list=[]
        for time in full_time_list:
            if time>=time_start:
                time_list.append(time)
            if len(time_list)==time_len:
                break
        value_list=full_value_list[full_time_list.index(time_list[0]):\
                                   full_time_list.index(time_list[-1])+1]
        time_frame=pd.DataFrame({'ture_value':value_list},index=time_list)
        return time_frame
    
    def get_observe_frame(self):
    '''
    Désagrégation de la charge active
    En se basant sur les états cachés à chaque instant pour chaque chaîne HMM obtenus précédemment, calculer une valeur extrême conditionnelle. La condition est basée sur la contrainte totale de la puissance active, précédemment synthétisée, et l'extrême correspond à la maximisation du produit des probabilités d'observation de la puissance active pour tous les appareils dans un état caché spécifique.
    '''
    self.observe_frame=pd.DataFrame(np.empty((len(self.aggregate_frame.index),self.HMMs_num)),\
                                    index=list(self.aggregate_frame.index),\
                                    columns=list(self.states_frame.columns))
    for i in list(self.aggregate_frame.index):
        guass_func_list=[]
        x0=[]
        e=1e-10
        for j in range(2,self.HMMs_num+2):
            state=int(self.states_frame.loc[i,'channel_'+str(j)])
            meanlist=list(self.HMMs_para['channel_'+str(j)][2].reshape(1,-1)[0])
            mean=sorted(meanlist)[state-1]
            covar=self.HMMs_para['channel_'+str(j)][3].reshape(1,-1)[0][meanlist.index(mean)]
            guass_func_list.append(norm(loc=mean, scale=covar**0.5))
            x0.append(mean)
        fun=lambda o:(guass_func_list[0].logpdf(o[0])+guass_func_list[1].logpdf(o[1])
                     +guass_func_list[2].logpdf(o[2])
                     +guass_func_list[3].logpdf(o[3])
                     +guass_func_list[4].logpdf(o[4])
                     #+guass_func_list[5].logpdf(o[5])
                     )**-1
        cons=({'type':'eq','fun':lambda o: o[0]+o[1]\
               +o[2]\
               +o[3]\
               +o[4]\
               #+o[5]\
               -self.aggregate_frame.loc[i,'aggregate_value']},
               {'type':'ineq','fun':lambda o:o[0]-e},
               {'type':'ineq','fun':lambda o:o[1]-e},
               {'type':'ineq','fun':lambda o:o[2]-e},
               {'type':'ineq','fun':lambda o:o[3]-e},
               {'type':'ineq','fun':lambda o:o[4]-e},
               #{'type':'ineq','fun':lambda o:o[5]-e}
               )
        res=minimize(fun, x0, method='SLSQP', constraints=cons)
        for k in range(2,self.HMMs_num+2):
            self.observe_frame.loc[i,'channel_'+str(k)]=res.x[k-2]
            
            
    
                
def compute_F1_measure(self):
    '''
Calculer l'indicateur de précision de désagrégation de charge F1_measure ; plus il est proche de 1, plus la précision de la désagrégation est élevée.
    '''
    pre_list=[]
    rec_list=[]
    F1_rspectively=[]
    for i in range(2,self.HMMs_num+2):
        tp=[]
        ture_values=[]
        predict_values=[]
        for j in range(len(self.observe_frame.index)):
            tp.append(min(self.observe_frame.values[j,i-2],\
                          self.true_value_frame_dic['channel_'+str(i)].values[j,0]))
            ture_values.append(self.true_value_frame_dic['channel_'+str(i)].values[j,0])
            predict_values.append(self.observe_frame.values[j,i-2])
        pre_i=sum(tp)/sum(predict_values)
        if set(ture_values)==set([0]):
            rec_i=1
        else:
            rec_i=sum(tp)/sum(ture_values)
        f1_i=2*pre_i*rec_i/(pre_i+rec_i)
        F1_rspectively.append(f1_i)
        pre_list.append(pre_i)
        rec_list.append(rec_i)
    pre=np.mean(pre_list)
    rec=np.mean(rec_list)
    F1_general=2*pre*rec/(pre+rec)
    return F1_general,F1_rspectively
        
    
    
def compute_NDE(self):
    '''
    Calculer l'indicateur d'erreur normalisée de désagrégation (NDE) pour évaluer la précision de la désagrégation de charge ; 
    plus il est proche de 0, meilleure est la précision de la désagrégation.
    '''
    subtract_squre_i=[]
    predict_squre_i=[]
    NDE_respectively=[]
    for i in range(2,self.HMMs_num+2):
        subtract_squre_t=[]
        predict_squre_t=[]
        for j in range(len(self.observe_frame.index)):
            subtract_squre_t.append((self.observe_frame.values[j,i-2]-\
                                       self.true_value_frame_dic['channel_'+str(i)].values[j,0])**2)
            predict_squre_t.append((self.observe_frame.values[j,i-2])**2)
        subtract_squre_t_sum=sum(subtract_squre_t)
        predict_squre_t_sum=sum(predict_squre_t)
        NDE_respectively.append(np.sqrt(subtract_squre_t_sum/predict_squre_t_sum))
        subtract_squre_i.append(subtract_squre_t_sum)
        predict_squre_i.append(predict_squre_t_sum)
    NDE_general=np.sqrt(sum(subtract_squre_i)/sum(predict_squre_i))
    return NDE_general,NDE_respectively
        
def energy_consumption_portion(self):
    aggregate_consumption=0
    true_indiv_consumption_protion=[]
    observe_indiv_consumption_protion=[]
    for i in range(len(self.aggregate_frame.index)):
        if i<len(self.aggregate_frame.index)-1:
            aggregate_consumption+=self.aggregate_frame.values[i,0]*(self.aggregate_frame.index[i+1]-\
                                                          self.aggregate_frame.index[i])
    for j in range(2,self.HMMs_num+2):
        true_indiv_aggregate=0
        observe_indiv_aggregate=0
        for k in range(len(self.aggregate_frame.index)):
            if k<len(self.aggregate_frame.index)-1:
                true_indiv_aggregate+=self.true_value_frame_dic['channel_'+str(j)].values[k,0]*\
                                      (self.true_value_frame_dic['channel_'+str(j)].index[k+1]-\
                                       self.true_value_frame_dic['channel_'+str(j)].index[k])
                observe_indiv_aggregate+=self.observe_frame.values[k,j-2]*\
                                        (self.observe_frame.index[k+1]-\
                                         self.observe_frame.index[k])
        true_indiv_consumption_protion.append(true_indiv_aggregate/aggregate_consumption) 
        observe_indiv_consumption_protion.append(observe_indiv_aggregate/aggregate_consumption)
    return true_indiv_consumption_protion,observe_indiv_consumption_protion
       
    
def compute_MAE(self):
    err_list=[]
    MAE_list=[]
    for i in range(2,self.HMMs_num+2):
        for j in range(len(self.observe_frame.index)):
            err_list.append(abs(self.true_value_frame_dic['channel_'+str(i)].values[j,0]-\
                                                      self.observe_frame.values[j,i-2]))
        MAE_list.append(np.mean(err_list[-len(self.observe_frame.index):]))
    return np.mean(err_list),MAE_list
    
def compute_RMSE(self):
    err_squre_list=[]
    RMSE_list=[]
    for i in range(2,self.HMMs_num+2):
        for j in range(len(self.observe_frame.index)):
            err_squre_list.append((self.true_value_frame_dic['channel_'+str(i)].values[j,0]-\
                                                      self.observe_frame.values[j,i-2])**2)
        RMSE_list.append((np.mean(err_squre_list[-len(self.observe_frame.index):]))**0.5)
    return (np.mean(err_squre_list))**0.5,RMSE_list
        
            

'''
Lors de l'utilisation de cette classe, modifier d'abord le nombre de variables dans la fonction objectif et les contraintes en fonction de la valeur de HMMs_num.

'''  

house3_fhmm=factorial_hmm(HMMs_num=5,house=6)
house3_fhmm.get_HMMs_para()
house3_fhmm.compute_FHMM_para()
house3_fhmm.get_aggregate_frame(time_start=1.40577e9,time_len=5e3)
house3_fhmm.decode()
house3_fhmm.get_observe_frame()
print(house3_fhmm.aggregate_frame)
print(house3_fhmm.states_frame)
print(house3_fhmm.observe_frame)
print(house3_fhmm.n_components)


for i in range(2,house3_fhmm.HMMs_num+2):
    GMT_line=[]
    time_num=list(house3_fhmm.observe_frame.index)
    for j in time_num:
        GMT_stamp=int(j)-7*3600
        GMT_time=time.localtime(GMT_stamp)
        GMT=time.strftime('%m%d%H%M%S',GMT_time)
        GMT_line.append(GMT)
    
    x=[datetime.strptime(d,'%m%d%H%M%S') for d in GMT_line]
    
    fig=plt.figure()
    ax1=fig.add_subplot(1,1,1)
    ax1.plot(x,list(house3_fhmm.observe_frame['channel_'+str(i)]),label='disaggragate')
    plt.gcf().autofmt_xdate()
    
    allhours=mdates.HourLocator()
    ax1.xaxis.set_major_locator(allhours) 
    ax1.xaxis.set_major_formatter(mdates.DateFormatter('%m%d%H'))
    
    minutesLoc=mpl.dates.MinuteLocator(interval=30)
    ax1.xaxis.set_minor_locator(minutesLoc)
    ax1.xaxis.set_minor_formatter(mdates.DateFormatter('%M'))
    
    ax1.tick_params(pad=15)
    
    mpl.rcParams['xtick.direction'] = 'in'
    mpl.rcParams['ytick.direction'] = 'in'
    
    ax1.set_xlabel('date(month/day/hour/minute)')
    ax1.set_ylabel('active power(W)')
    
    plt.legend()
    plt.show()


    fig=plt.figure()
    ax1=fig.add_subplot(1,1,1)
    ax1.plot(x,list(house3_fhmm.true_value_frame_dic['channel_'+str(i)]['ture_value']),label='real_data')
    plt.gcf().autofmt_xdate()
    
    allhours=mdates.HourLocator()
    ax1.xaxis.set_major_locator(allhours) 
    ax1.xaxis.set_major_formatter(mdates.DateFormatter('%m%d%H'))
    
    minutesLoc=mpl.dates.MinuteLocator(interval=30)
    ax1.xaxis.set_minor_locator(minutesLoc)
    ax1.xaxis.set_minor_formatter(mdates.DateFormatter('%M'))
    
    ax1.tick_params(pad=15)
    
    mpl.rcParams['xtick.direction'] = 'in'
    mpl.rcParams['ytick.direction'] = 'in'
    
    ax1.set_xlabel('date(month/day/hour/minute)')
    ax1.set_ylabel('active power(W)')
    
    plt.legend()
    plt.show()
    
print(house3_fhmm.compute_F1_measure())
print(house3_fhmm.compute_NDE())
print(house3_fhmm.energy_consumption_portion())

        
        
        

IndentationError: expected an indented block (2609529065.py, line 227)

In [ ]:
import hmmlearn.hmm as hmm
import numpy as np
import itertools
import pandas as pd
from scipy.stats import norm
from scipy.optimize import minimize
from docx import Document
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.dates as mdates
from datetime import datetime
import time

def read_clusters(file_path):
    doc=Document(file_path)
    cluster_num=len(doc.tables[3].rows)-1
    clusters=[]
    for i in range(1,len(doc.tables[3].rows)):
        clusters.append(float(doc.tables[3].cell(i,1).text))
    return (cluster_num,clusters)
 
    
def read_dataset(file_path,require_time=None):
    f=open(file_path,'r')
    value=[]
    time_line=[]
    line=f.readline()
    while line:
        data=line.split(' ')
        value.append(int(data[1][0:len(data[1])-1]))
        time_line.append(int(data[0]))
        line=f.readline()
    f.close()
    if require_time==None:
        return value
    else:
        return value,time_line
    
       

class factorial_hmm(object):
    def __init__(self,HMMs_num,house):
        self.HMMs_num=HMMs_num   # Nombre de chaînes de Markov cachées, c'est-à-dire le nombre total d'appareils
        self.house=house         # L'ensemble des données correspond au numéro de série de la pièce
        self.HMMs_para={}        # Dictionnaire contenant chaque paramètre HMM
        self.states_combination=[]  # Chaque tuple d'état hmm, de la forme ('1', '2', '3', '4',...) La longueur de ce n-uplet est le nombre d'états cachés hmm
        self.startprob_=None      # fhmm distribution de l'état initial
        self.transmat_=None       # Matrice de transfert d'état
        self.means_=None          # fhmm matrice moyenne de la densité de probabilité de l'état observé
        self.covars_=None         # matrice de covariance de la densité de probabilité de l'état observé
        self.n_components=None    # Nombre total d'états cachés #fhmm
        self.states_frame=None    # Tableau d'identification des états cachés pour chaque hmm
        self.observe_frame=None   # Tableaux de décomposition de la puissance active des charges hmm pour chaque barre
        self.aggregate_frame=None # Tableaux de données échantillonnées de la puissance active totale synthétisée sur une période donnée
        self.true_value_frame_dic={} # Dictionnaire composé d'un tableau de données d'échantillonnage actives réelles pour chaque appareil sur une période donnée, indexées par canal.
         
    def get_HMMs_para(self):
        '''
        En utilisant les données d'échantillonnage actif de chaque appareil électrique et le nombre d'états pour chaque appareil obtenu à partir du clustering      
        précédent, nous apprenons et obtenons les paramètres HMM pour chaque canal. Les résultats sont stockés dans un dictionnaire appelé self.HMMs_para o
        ù la clé est le canal et la valeur est une liste contenant successivement pi, A, la moyenne, et la covariance.
        
        '''
        for j in range(2,self.HMMs_num+2):
            cluster_path='C:/Users/lenovo/Desktop/domestic UK2015/house'+str(self.house)+'/channel_'+str(j)+'.docx'
            states_num,means=read_clusters(cluster_path)
            dataset_path='C:/Users/lenovo/Desktop/house'+str(self.house)+'/channel_'+str(j)+'.dat'
            sample_value=read_dataset(dataset_path)
            HMM_model=hmm.GaussianHMM(n_components=states_num, covariance_type='full')
            HMM_model.fit(np.array([sample_value]).reshape(-1,1))
            self.HMMs_para['channel_'+str(j)]=[HMM_model.startprob_,HMM_model.transmat_,\
                           HMM_model.means_,HMM_model.covars_]
            
    def compute_FHMM_para(self):
        '''
        Appel à la fonction pour calculer les différents paramètres de chaque FHMM et stocker les résultats. 
        Le nombre de colonnes pour pi, l'ordre de A, le nombre d'éléments pour les moyennes et les covariances s
        ont tous égaux au nombre d'états cachés, c'est-à-dire self.n_components. L'ordre d'agencement de tous les paramètres correspond à 
        l'ordre d'agencement des combinaisons d'états dans la variable self.states_combination.

        '''
        self.startprob_=self.compute_pi_fhmm()
        self.transmat_=self.compute_A_fhmm()
        self.means_=self.compute_means_fhmm()
        self.covars_=self.compute_covars_fhmm()
        self.n_components=self.compute_n_components()
        
    def compute_pi_fhmm(self):
        '''
        Utiliser le produit de Kronecker pour calculer la matrice pi du modèle FHMM.
        '''
        list_pi=[]
        for j in range(2,len(self.HMMs_para.keys())+2):
            list_pi.append(self.HMMs_para['channel_'+str(j)][0])
        result = list_pi[0]
        for i in range(len(list_pi) - 1):
            result = np.kron(result, list_pi[i + 1])
        return result
    
    def compute_A_fhmm(self):
        '''
       Utiliser le produit de Kronecker pour calculer la matrice A du modèle FHMM.
        '''
        list_A=[]
        for j in range(2,len(self.HMMs_para.keys())+2):
            list_A.append(self.HMMs_para['channel_'+str(j)][1])
        result = list_A[0]
        for i in range(len(list_A) - 1):
            result = np.kron(result, list_A[i + 1])
        return result
    
    def compute_means_fhmm(self):
        '''
        En utilisant la méthode itertools.product, combiner les moyennes de chaque état caché de chaque HMM du FHMM. 
        Ensuite, selon la propriété d'indépendance mutuelle de chaque HMM, additionner ces moyennes pour obtenir la moyenne de la distribution d'observation 
        pour chaque état caché du FHMM.

        En fonction de la liste des moyennes pour chaque HMM, générer une chaîne de chiffres débutant à '1' dans le même ordre que ces moyennes, 
        où la position de la moyenne la plus petite correspond au chiffre '1', et ainsi de suite. En utilisant ensuite la méthode itertools.product 
        pour combiner chaque chaîne de chiffres correspondant à la liste des moyennes, obtenir states_combination. 
        Cette variable de type liste a une longueur égale au nombre d'états cachés du FHMM. Chaque élément représente un état caché du FHMM, 
        et chaque élément est un tuple composé de plusieurs caractères numériques individuels. La longueur du tuple est égale au nombre de chaînes HMM, 
        et cela représente quelles moyennes de quelle chaîne HMM composent cet état caché. L'ordre des éléments dans states_combination correspond à 
        l'ordre des éléments dans pi, A, moyennes et covariances."
        '''
        list_means=[]
        result=[]
        for j in range(2,len(self.HMMs_para.keys())+2):
            list_means.append(self.HMMs_para['channel_'+str(j)][2].reshape(1,-1)[0])
        for i in itertools.product(*list_means):
            result.append(sum(i))
        states_number=[]
        for k in list_means:
            string=''
            sort_means=sorted(k)
            for p in k:
                for t in range(len(sort_means)):
                    if p==sort_means[t]:
                        string+=str(t+1)
                        break
            states_number.append(string)
        for q in itertools.product(*states_number):
            self.states_combination.append(q)
        return np.array([result]).reshape(-1,1)
    
    def compute_covars_fhmm(self):
        '''
        Calculer la matrice de covariance de fhmm
        '''
        list_covars=[]
        result=[]
        for j in range(2,len(self.HMMs_para.keys())+2):
            list_covars.append(self.HMMs_para['channel_'+str(j)][3].reshape(1,-1)[0])
        for i in itertools.product(*list_covars):
            result.append(sum(i))
        return np.array([result]).reshape(-1,1,1)

    def compute_n_components(self):
        '''
        Calculer le nombre d'états cachés fhmm
        '''
        return len(self.states_combination)
    
    def get_aggregate_frame(self,time_start,time_len):
        '''
        Obtenir la somme des valeurs réelles de puissance active de tous les appareils électriques pendant la plage de temps spécifiée par time_range. 
        Renvoyer un tableau (DataFrame) avec pour index les points temporels de cette période et les valeurs représentant la somme des puissances actives réelles. 
        Ce résultat servira de contrainte totale de puissance active pour la désagrégation de la charge électrique ultérieure et pour prédire les états cachés du modèle FHMM.
        '''
        for i in range(2,self.HMMs_num+2):
            dataset_path='C:/Users/lenovo/Desktop/house'+str(self.house)+'/channel_'+str(i)+'.dat'
            sample_value,sample_time=read_dataset(file_path=dataset_path,require_time='yes')
            time_frame=self.get_a_time_frame(time_start,time_len,sample_time,sample_value)
            if i!=2:
                time_frame.index=self.true_value_frame_dic['channel_2'].index
            self.true_value_frame_dic['channel_'+str(i)]=time_frame
    
        aggregate_value=[]
        for j in range(int(time_len)):
            value=0
            for k in range(2,self.HMMs_num+2):
                value+=self.true_value_frame_dic['channel_'+str(k)].values[j,0]
            aggregate_value.append(value)
        self.aggregate_frame=pd.DataFrame(np.array([aggregate_value]).reshape(-1,1),\
                                          index=list(self.true_value_frame_dic['channel_2'].index),columns=['aggregate_value'])
        
    
    def decode(self):
        '''
        En utilisant la somme précédemment calculée des valeurs totales d'énergie active sur une période de temps donnée ainsi que les paramètres synthétiques du FHMM, 
        prédire la séquence d'états cachés du FHMM. Ensuite, selon la correspondance correspondante, organiser les éléments de self.states_combination 
        pour décomposer les états cachés de chaque HMM à chaque instant. Renvoyer un tableau d'états où les index sont les moments dans cette période et 
        les colonnes représentent les canaux.
        '''
        fhmm_model=hmm.GaussianHMM(n_components=self.n_components, covariance_type='full')
        fhmm_model.startprob_=self.startprob_
        fhmm_model.transmat_=self.transmat_
        fhmm_model.means_=self.means_
        fhmm_model.covars_=self.covars_
        logprob,state_sequence=fhmm_model.decode(self.aggregate_frame.values)
        state_sequence_str={}
        for i in range(2,self.HMMs_num+2):
            state_str=[]
            for j in state_sequence:
                state_str.append(int(self.states_combination[j][i-2]))
            state_sequence_str['channel_'+str(i)]=state_str
        self.states_frame=pd.DataFrame(state_sequence_str,index=list(self.aggregate_frame.index))
        
        
    def get_a_time_frame(self,time_start,time_len,full_time_list,full_value_list):
        '''
        Intercepter le point temporel et la valeur d'échantillonnage active 
        correspondante en fonction de l'intervalle de temps, renvoyer une trame.
        '''
        time_list=[]
        for time in full_time_list:
            if time>=time_start:
                time_list.append(time)
            if len(time_list)==time_len:
                break
        value_list=full_value_list[full_time_list.index(time_list[0]):\
                                   full_time_list.index(time_list[-1])+1]
        time_frame=pd.DataFrame({'ture_value':value_list},index=time_list)
        return time_frame


def get_observe_frame(self):
    '''
    Désagrégation de la charge active
    En se basant sur les états cachés à chaque instant pour chaque chaîne HMM obtenus précédemment, calculer une valeur extrême conditionnelle. La condition est basée sur la contrainte totale de la puissance active, précédemment synthétisée, et l'extrême correspond à la maximisation du produit des probabilités d'observation de la puissance active pour tous les appareils dans un état caché spécifique.
    '''
    self.observe_frame=pd.DataFrame(np.empty((len(self.aggregate_frame.index),self.HMMs_num)),\
                                    index=list(self.aggregate_frame.index),\
                                    columns=list(self.states_frame.columns))
    for i in list(self.aggregate_frame.index):
        guass_func_list=[]
        x0=[]
        e=1e-10
        for j in range(2,self.HMMs_num+2):
            state=int(self.states_frame.loc[i,'channel_'+str(j)])
            meanlist=list(self.HMMs_para['channel_'+str(j)][2].reshape(1,-1)[0])
            mean=sorted(meanlist)[state-1]
            covar=self.HMMs_para['channel_'+str(j)][3].reshape(1,-1)[0][meanlist.index(mean)]
            guass_func_list.append(norm(loc=mean, scale=covar**0.5))
            x0.append(mean)
        fun=lambda o:(guass_func_list[0].logpdf(o[0])+guass_func_list[1].logpdf(o[1])
                     +guass_func_list[2].logpdf(o[2])
                     +guass_func_list[3].logpdf(o[3])
                     +guass_func_list[4].logpdf(o[4])
                     #+guass_func_list[5].logpdf(o[5])
                     )**-1
        cons=({'type':'eq','fun':lambda o: o[0]+o[1]\
               +o[2]\
               +o[3]\
               +o[4]\
               #+o[5]\
               -self.aggregate_frame.loc[i,'aggregate_value']},
               {'type':'ineq','fun':lambda o:o[0]-e},
               {'type':'ineq','fun':lambda o:o[1]-e},
               {'type':'ineq','fun':lambda o:o[2]-e},
               {'type':'ineq','fun':lambda o:o[3]-e},
               {'type':'ineq','fun':lambda o:o[4]-e},
               #{'type':'ineq','fun':lambda o:o[5]-e}
               )
        res=minimize(fun, x0, method='SLSQP', constraints=cons)
        for k in range(2,self.HMMs_num+2):
            self.observe_frame.loc[i,'channel_'+str(k)]=res.x[k-2]
            
            
    
                
def compute_F1_measure(self):
    '''
Calculer l'indicateur de précision de désagrégation de charge F1_measure ; plus il est proche de 1, plus la précision de la désagrégation est élevée.
    '''
    pre_list=[]
    rec_list=[]
    F1_rspectively=[]
    for i in range(2,self.HMMs_num+2):
        tp=[]
        ture_values=[]
        predict_values=[]
        for j in range(len(self.observe_frame.index)):
            tp.append(min(self.observe_frame.values[j,i-2],\
                          self.true_value_frame_dic['channel_'+str(i)].values[j,0]))
            ture_values.append(self.true_value_frame_dic['channel_'+str(i)].values[j,0])
            predict_values.append(self.observe_frame.values[j,i-2])
        pre_i=sum(tp)/sum(predict_values)
        if set(ture_values)==set([0]):
            rec_i=1
        else:
            rec_i=sum(tp)/sum(ture_values)
        f1_i=2*pre_i*rec_i/(pre_i+rec_i)
        F1_rspectively.append(f1_i)
        pre_list.append(pre_i)
        rec_list.append(rec_i)
    pre=np.mean(pre_list)
    rec=np.mean(rec_list)
    F1_general=2*pre*rec/(pre+rec)
    return F1_general,F1_rspectively
        
    
    
def compute_NDE(self):
    '''
    Calculer l'indicateur d'erreur normalisée de désagrégation (NDE) pour évaluer la précision de la désagrégation de charge ; 
    plus il est proche de 0, meilleure est la précision de la désagrégation.
    '''
    subtract_squre_i=[]
    predict_squre_i=[]
    NDE_respectively=[]
    for i in range(2,self.HMMs_num+2):
        subtract_squre_t=[]
        predict_squre_t=[]
        for j in range(len(self.observe_frame.index)):
            subtract_squre_t.append((self.observe_frame.values[j,i-2]-\
                                       self.true_value_frame_dic['channel_'+str(i)].values[j,0])**2)
            predict_squre_t.append((self.observe_frame.values[j,i-2])**2)
        subtract_squre_t_sum=sum(subtract_squre_t)
        predict_squre_t_sum=sum(predict_squre_t)
        NDE_respectively.append(np.sqrt(subtract_squre_t_sum/predict_squre_t_sum))
        subtract_squre_i.append(subtract_squre_t_sum)
        predict_squre_i.append(predict_squre_t_sum)
    NDE_general=np.sqrt(sum(subtract_squre_i)/sum(predict_squre_i))
    return NDE_general,NDE_respectively
        
def energy_consumption_portion(self):
    aggregate_consumption=0
    true_indiv_consumption_protion=[]
    observe_indiv_consumption_protion=[]
    for i in range(len(self.aggregate_frame.index)):
        if i<len(self.aggregate_frame.index)-1:
            aggregate_consumption+=self.aggregate_frame.values[i,0]*(self.aggregate_frame.index[i+1]-\
                                                          self.aggregate_frame.index[i])
    for j in range(2,self.HMMs_num+2):
        true_indiv_aggregate=0
        observe_indiv_aggregate=0
        for k in range(len(self.aggregate_frame.index)):
            if k<len(self.aggregate_frame.index)-1:
                true_indiv_aggregate+=self.true_value_frame_dic['channel_'+str(j)].values[k,0]*\
                                      (self.true_value_frame_dic['channel_'+str(j)].index[k+1]-\
                                       self.true_value_frame_dic['channel_'+str(j)].index[k])
                observe_indiv_aggregate+=self.observe_frame.values[k,j-2]*\
                                        (self.observe_frame.index[k+1]-\
                                         self.observe_frame.index[k])
        true_indiv_consumption_protion.append(true_indiv_aggregate/aggregate_consumption) 
        observe_indiv_consumption_protion.append(observe_indiv_aggregate/aggregate_consumption)
    return true_indiv_consumption_protion,observe_indiv_consumption_protion
       
    
def compute_MAE(self):
    err_list=[]
    MAE_list=[]
    for i in range(2,self.HMMs_num+2):
        for j in range(len(self.observe_frame.index)):
            err_list.append(abs(self.true_value_frame_dic['channel_'+str(i)].values[j,0]-\
                                                      self.observe_frame.values[j,i-2]))
        MAE_list.append(np.mean(err_list[-len(self.observe_frame.index):]))
    return np.mean(err_list),MAE_list
    
def compute_RMSE(self):
    err_squre_list=[]
    RMSE_list=[]
    for i in range(2,self.HMMs_num+2):
        for j in range(len(self.observe_frame.index)):
            err_squre_list.append((self.true_value_frame_dic['channel_'+str(i)].values[j,0]-\
                                                      self.observe_frame.values[j,i-2])**2)
        RMSE_list.append((np.mean(err_squre_list[-len(self.observe_frame.index):]))**0.5)
    return (np.mean(err_squre_list))**0.5,RMSE_list
        
            

'''
Lors de l'utilisation de cette classe, modifier d'abord le nombre de variables dans la fonction objectif et les contraintes en fonction de la valeur de HMMs_num.

'''  

house3_fhmm=factorial_hmm(HMMs_num=5,house=6)
house3_fhmm.get_HMMs_para()
house3_fhmm.compute_FHMM_para()
house3_fhmm.get_aggregate_frame(time_start=1.40577e9,time_len=5e3)
house3_fhmm.decode()
house3_fhmm.get_observe_frame()
print(house3_fhmm.aggregate_frame)
print(house3_fhmm.states_frame)
print(house3_fhmm.observe_frame)
print(house3_fhmm.n_components)


for i in range(2,house3_fhmm.HMMs_num+2):
    GMT_line=[]
    time_num=list(house3_fhmm.observe_frame.index)
    for j in time_num:
        GMT_stamp=int(j)-7*3600
        GMT_time=time.localtime(GMT_stamp)
        GMT=time.strftime('%m%d%H%M%S',GMT_time)
        GMT_line.append(GMT)
    
    x=[datetime.strptime(d,'%m%d%H%M%S') for d in GMT_line]
    
    fig=plt.figure()
    ax1=fig.add_subplot(1,1,1)
    ax1.plot(x,list(house3_fhmm.observe_frame['channel_'+str(i)]),label='disaggragate')
    plt.gcf().autofmt_xdate()
    
    allhours=mdates.HourLocator()
    ax1.xaxis.set_major_locator(allhours) 
    ax1.xaxis.set_major_formatter(mdates.DateFormatter('%m%d%H'))
    
    minutesLoc=mpl.dates.MinuteLocator(interval=30)
    ax1.xaxis.set_minor_locator(minutesLoc)
    ax1.xaxis.set_minor_formatter(mdates.DateFormatter('%M'))
    
    ax1.tick_params(pad=15)
    
    mpl.rcParams['xtick.direction'] = 'in'
    mpl.rcParams['ytick.direction'] = 'in'
    
    ax1.set_xlabel('date(month/day/hour/minute)')
    ax1.set_ylabel('active power(W)')
    
    plt.legend()
    plt.show()


    fig=plt.figure()
    ax1=fig.add_subplot(1,1,1)
    ax1.plot(x,list(house3_fhmm.true_value_frame_dic['channel_'+str(i)]['ture_value']),label='real_data')
    plt.gcf().autofmt_xdate()
    
    allhours=mdates.HourLocator()
    ax1.xaxis.set_major_locator(allhours) 
    ax1.xaxis.set_major_formatter(mdates.DateFormatter('%m%d%H'))
    
    minutesLoc=mpl.dates.MinuteLocator(interval=30)
    ax1.xaxis.set_minor_locator(minutesLoc)
    ax1.xaxis.set_minor_formatter(mdates.DateFormatter('%M'))
    
    ax1.tick_params(pad=15)
    
    mpl.rcParams['xtick.direction'] = 'in'
    mpl.rcParams['ytick.direction'] = 'in'
    
    ax1.set_xlabel('date(month/day/hour/minute)')
    ax1.set_ylabel('active power(W)')
    
    plt.legend()
    plt.show()
    
print(house3_fhmm.compute_F1_measure())
print(house3_fhmm.compute_NDE())
print(house3_fhmm.energy_consumption_portion())
